In [172]:
!pip install ipython-sql

In [173]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [193]:
%sql sqlite://

In [195]:
%%sql
CREATE TABLE IF NOT EXISTS sales
(
    key       varchar(6),
    ts        timestamp,
    product   integer,
    completed boolean,
    price     float
);
INSERT INTO sales
VALUES ('sale_1', '2019-11-08 00:00', 0, TRUE, 1.1),
       ('sale_2', '2019-11-08 01:00', 0, FALSE, 1.2),
       ('sale_3', '2019-11-08 01:00', 0, TRUE, 1.3),
       ('sale_4', '2019-11-08 01:00', 1, FALSE, 1.4),
       ('sale_5', '2019-11-08 02:00', 1, TRUE, 1.5),
       ('sale_6', '2019-11-08 02:00', 1, TRUE, 1.5);

 * sqlite://
Done.
6 rows affected.


[]

In [196]:
%%sql
select * from sales;

 * sqlite://
Done.


key,ts,product,completed,price
sale_1,2019-11-08 00:00,0,1,1.1
sale_2,2019-11-08 01:00,0,0,1.2
sale_3,2019-11-08 01:00,0,1,1.3
sale_4,2019-11-08 01:00,1,0,1.4
sale_5,2019-11-08 02:00,1,1,1.5
sale_6,2019-11-08 02:00,1,1,1.5


##### Using Row Number and Rank Function to Check if there are any duplicates or get the Max or Min record

- In the above example because we have 3 products with sold on the same day with different Time. So if we want to consider the First order or last Order we can use Row Number Function
- Rank Funtion can be used to Rank the Product based on the time and price. 

In [197]:
%%sql
select key,ts,product, price,
row_number() over (partition by product order by ts) as row_num,
rank() over (partition by product order by ts) as rank_ts,
rank() over (partition by product order by ts, price) as rank_ts_price
from sales

 * sqlite://
Done.


key,ts,product,price,row_num,rank_ts,rank_ts_price
sale_1,2019-11-08 00:00,0,1.1,1,1,1
sale_2,2019-11-08 01:00,0,1.2,2,2,2
sale_3,2019-11-08 01:00,0,1.3,3,2,3
sale_4,2019-11-08 01:00,1,1.4,1,1,1
sale_5,2019-11-08 02:00,1,1.5,2,2,2
sale_6,2019-11-08 02:00,1,1.5,3,2,2


In [198]:
%%sql
select * from (
select key,ts,product, price, row_number() over (partition by product order by ts) as row_num
from sales)
where row_num = 1;

 * sqlite://
Done.


key,ts,product,price,row_num
sale_1,2019-11-08 00:00,0,1.1,1
sale_4,2019-11-08 01:00,1,1.4,1


#### LEAD and LAG functions to check the Forward and the Backward data.

In [199]:
%%sql
select *,
lag(price) over(partition by product) as lag,
lead(price) over(partition by product) as lead
from sales;

 * sqlite://
Done.


key,ts,product,completed,price,lag,lead
sale_1,2019-11-08 00:00,0,1,1.1,None,1.2
sale_2,2019-11-08 01:00,0,0,1.2,1.1,1.3
sale_3,2019-11-08 01:00,0,1,1.3,1.2,None
sale_4,2019-11-08 01:00,1,0,1.4,None,1.5
sale_5,2019-11-08 02:00,1,1,1.5,1.4,1.5
sale_6,2019-11-08 02:00,1,1,1.5,1.5,None


#### AVG function

In [200]:
%%sql
select *,
Round(avg(price) over(),2) as Avg,
Round(avg(price) over(partition by product),2) as Avg_prod,
Round(avg(price) over(partition by product order by completed),2) as Avg
from sales

 * sqlite://
Done.


key,ts,product,completed,price,Avg,Avg_prod,Avg_1
sale_2,2019-11-08 01:00,0,0,1.2,1.33,1.2,1.2
sale_1,2019-11-08 00:00,0,1,1.1,1.33,1.2,1.2
sale_3,2019-11-08 01:00,0,1,1.3,1.33,1.2,1.2
sale_4,2019-11-08 01:00,1,0,1.4,1.33,1.47,1.4
sale_5,2019-11-08 02:00,1,1,1.5,1.33,1.47,1.47
sale_6,2019-11-08 02:00,1,1,1.5,1.33,1.47,1.47


In [181]:
%%sql
CREATE TABLE IF NOT EXISTS "employee" (
    "id" INT,
    "first_name" TEXT,
    "last_name" TEXT,
    "department_id" INT,
    "salary" INT,
    "years_worked" INT
);
INSERT INTO "employee" VALUES
    (1,'Diane','Turner',1,5330,4),
    (2,'Clarence','Robinson',1,3617,2),
    (3,'Eugene','Phillips',1,4877,2),
    (4,'Philip','Mitchell',1,5259,3),
    (5,'Ann','Wright',2,2094,5),
    (6,'Charles','Wilson',2,5167,5),
    (7,'Russell','Johnson',2,3762,4),
    (8,'Jacqueline','Cook',2,6923,3),
    (9,'Larry','Lee',3,2796,4),
    (10,'Willie','Patterson',3,4771,5),
    (11,'Janet','Ramirez',3,3782,2),
    (12,'Doris','Bryant',3,6419,1),
    (13,'Amy','Williams',3,6261,1),
    (14,'Keith','Scott',3,4928,8),
    (15,'Karen','Morris',4,6347,6),
    (16,'Kathy','Sanders',4,6286,1),
    (17,'Joe','Thompson',5,5639,3),
    (18,'Barbara','Clark',5,3232,1),
    (19,'Todd','Bell',5,4653,1),
    (20,'Ronald','Butler',5,2076,5);

CREATE TABLE IF NOT EXISTS "department" (
    "id" INT,
    "name" TEXT
);
INSERT INTO "department" VALUES
    (1,'IT'),
    (2,'Management'),
    (3,'Human Resources'),
    (4,'Accounting'),
    (5,'Help Desk');
    
CREATE TABLE IF NOT EXISTS "purchase" (
    "id" INT,
    "department_id" INT,
    "item" TEXT,
    "price" INT
);
INSERT INTO "purchase" VALUES
    (1,4,'monitor',531),
    (2,1,'printer',315),
    (3,3,'whiteboard',170),
    (4,5,'training',117),
    (5,3,'computer',2190),
    (6,1,'monitor',418),
    (7,3,'whiteboard',120),
    (8,3,'monitor',388),
    (9,5,'paper',37),
    (10,1,'paper',695),
    (11,3,'projector',407),
    (12,4,'garden party',986),
    (13,5,'projector',481),
    (14,2,'chair',180),
    (15,2,'desk',854),
    (16,2,'post-it',15),
    (17,3,'paper',60),
    (18,2,'tv',943),
    (19,2,'desk',478),
    (20,5,'keyboard',214);


 * sqlite://
Done.
20 rows affected.
Done.
5 rows affected.
Done.
20 rows affected.


[]

In [201]:
%%sql
select Count(*) as CNT from employee

 * sqlite://
Done.


CNT
20


In [187]:
%%sql

SELECT
  e.id,d.id,
  first_name,
  last_name,
  department_id,
  salary
  MIN(salary) OVER(PARTITION BY department_id) as Min,
  MAX(salary) OVER(PARTITION BY department_id) as Max
FROM employee as e
left Join department as d on d.id = e.department_id 
where e.department_id In (1,3)


 * sqlite://
Done.


id,id_1,first_name,last_name,department_id,salary,row_num,Min,Max
1,1,Diane,Turner,1,5330,1,3617,5330
2,1,Clarence,Robinson,1,3617,1,3617,5330
3,1,Eugene,Phillips,1,4877,1,3617,5330
4,1,Philip,Mitchell,1,5259,1,3617,5330
9,3,Larry,Lee,3,2796,1,2796,6419
10,3,Willie,Patterson,3,4771,1,2796,6419
11,3,Janet,Ramirez,3,3782,1,2796,6419
12,3,Doris,Bryant,3,6419,1,2796,6419
13,3,Amy,Williams,3,6261,1,2796,6419
14,3,Keith,Scott,3,4928,1,2796,6419


In [184]:
%%sql
select *,
Avg(price) over () as Avg,
avg(price) over (partition by item) as Avg_item
from purchase
where department_id = 3;

 * sqlite://
Done.


id,department_id,item,price,Avg,Avg_item
5,3,computer,2190,555.8333333333334,2190.0
8,3,monitor,388,555.8333333333334,388.0
17,3,paper,60,555.8333333333334,60.0
11,3,projector,407,555.8333333333334,407.0
3,3,whiteboard,170,555.8333333333334,145.0
7,3,whiteboard,120,555.8333333333334,145.0


In [189]:
%%sql
select First_name, Last_name, Salary, d.name
from employee as e
Inner Join department as d on e.department_id=d.id
where department_id = 1;

 * sqlite://
Done.


first_name,last_name,salary,name
Clarence,Robinson,3617,IT
Diane,Turner,5330,IT
Eugene,Phillips,4877,IT
Philip,Mitchell,5259,IT
